In [ ]:
from tqdm import tqdm
import logging
from dfpp.transformation.sources.unicef_org.retrieve import (
    get_indicator,
    get_sdmx_client,
    get_dataflow_codebook,
)
from dfpp.transformation.sources.unicef_org.transform import transform
from dfpp.publishing import publish_series
from dfpp.geo_utils import get_iso3_to_official_name_map


logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(module)s - %(message)s",
    handlers=[logging.StreamHandler()],
)
ISO_3_MAP = await get_iso3_to_official_name_map()

In [ ]:
client = get_sdmx_client()
dataflow_id = "GLOBAL_DATAFLOW"
df_indicators, df_dimensions, df_attrributes = get_dataflow_codebook(
    client, dataflow_id=dataflow_id
)

In [ ]:
failed_series = []
indicators_processed = 0

parent_code = df_indicators[df_indicators.parent == ""].indicator_code.unique().tolist()

indicator_codes = df_indicators.indicator_code.unique().tolist()
print("Indicator codes to query:", len(indicator_codes))
for indicator_id in tqdm(indicator_codes):
    if indicator_id in parent_code:
        continue
    logging.info(indicator_id)
    try:
        df_source = get_indicator(
            client, dataflow_id=dataflow_id, indicator_id=indicator_id
        )
        if "REF_PERIOD" in df_source:
            logging.debug(f"{indicator_id} has REF_PERIOD")

        df = df_source.copy()
        df = transform(
            df_indicator=df,
            df_dimensions=df_dimensions,
            df_attributes=df_attrributes,
            df_indicators=df_indicators,
            iso_3_map=ISO_3_MAP,
        )
        publish_series(indicator_id, df)
        indicators_processed += 1
    except Exception as e:
        logging.error(indicator_id, str(e))
        failed_series.append((indicator_id, e))
    logging.info(f"error count: {len(failed_series)}")
    logging.info(f"indicators_processed: {indicators_processed}")

In [ ]:
assert len(failed_series) == 0, print(failed_series)